In [2]:
import torch
import torch.nn as nn
import numpy as np

In [3]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [4]:
x = np.array([2.0, 1.0, 0.1])
outputs = softmax(x)
print('softmax numpy: ', outputs)

softmax numpy:  [0.65900114 0.24243297 0.09856589]


In [5]:
x = torch.tensor([2.0, 1.0, 0.1])
outputs = torch.softmax(x, dim=0)
print('Softmax, torch: ', outputs)

Softmax, torch:  tensor([0.6590, 0.2424, 0.0986])


In [6]:
def cross_entropy(actual, predicted):
    EPS = 1e-15
    predicted = np.clip(predicted, EPS, 1 - EPS)
    loss = -np.sum(actual * np.log(predicted))
    return loss 

In [7]:
# Y must be one hot encoded
# if class 0: [1, 0, 0]
# if class 1: [0, 1, 0]
# if class 2: [0, 0, 1]

y = np.array([1, 0, 0])
y_pred_good  = np.array([0.7, 0.2, 0.1])
y_pred_bad = np.array([0.1, 0.3, 0.6])

l1 = cross_entropy(y, y_pred_good)
l2 = cross_entropy(y, y_pred_bad)

print(f"Loss1 numpy: {l1:.4f}")
print(f"Loss2 numpy: {l2:.4f}")


Loss1 numpy: 0.3567
Loss2 numpy: 2.3026


In [8]:
# CrossEntropyLoss in Pytorch (applies softmax)
# nn.LogSoftmax + nn.NLLLoss
# NLLLoss = Negative log likelihood loss
loss = nn.CrossEntropyLoss()
# loss(input, target)

In [9]:
# target is of size nSamples = 1
# each element has class labels: 0, 1 or 2
# y = (=target) contains class labels, not one-hot

Y = torch.tensor([0])

# input is of size nSamples x nClassses = 1 x 3
# y_pred (=input) must be raw, unnormalizes scores (logits) for each class, not softmax

y_pred_good = torch.tensor([[2.0, 1.0, 0.1]])
y_pred_bad = torch.tensor([[0.5, 0.2, 0.3]])

l1 = loss(y_pred_good, Y)
l2 = loss(y_pred_bad, Y)

print(f'Pytorch loss1: {l1.item():.4f}')
print(f'Pytorch loss2: {l2.item():.4f}')


Pytorch loss1: 0.4170
Pytorch loss2: 0.9398


In [10]:
# get predictions

_, predictions1 = torch.max(y_pred_good, 1)
_, predictions2 = torch.max(y_pred_bad, dim=1)

print(f'Actual class: {Y.item()}, y_pred1: {predictions1.item()}, y_pred2: {predictions2.item()}')

Actual class: 0, y_pred1: 0, y_pred2: 0


In [12]:
# Allow batch loss for multiple samples

# target is of size nBatch = 3
# each element has class label: 0, 1, or 2

Y = torch.tensor([2, 0, 1])

# input is of size nBatch x nClasses = 3 x 3
# y_pred are logits (not softmax)

y_pred_good = torch.tensor(
    [[0.1, 0.2, 3.9],
     [1.2, 0.1, 0.3],
     [0.3, 2.2, 0.2]]
)

y_pred_bad = torch.tensor(
    [[0.9, 0.2, 0.1],
     [0.1, 0.3, 1.5],
     [1.2, 0.2, 0.5]]
)

l1 = loss(y_pred_good, Y)
l2 = loss(y_pred_bad, Y)

print(f'Batch loss1: {l1.item():.4f}')
print(f'Batch loss2: {l2.item():.4f}')

# get predictions

_, predictions1 = torch.max(y_pred_good, dim=1)
_, predictions2 = torch.max(y_pred_bad, dim=1)
print(f'Actual class: {Y}, y_pred1: {predictions1}, y_pred2: {predictions2}')

Batch loss1: 0.2834
Batch loss2: 1.6418
Actual class: tensor([2, 0, 1]), y_pred1: tensor([2, 0, 1]), y_pred2: tensor([0, 2, 0])


# Binary classification

In [13]:
class NeuralNet1(nn.Module):

    def __init__(self, input_size, hidden_size):
        super(NeuralNet1, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # add sigmoid at the end
        y_pred = torch.sigmoid(out)
        return y_pred
    

model = NeuralNet1(input_size=28*28, hidden_size=5)
criterian = nn.BCELoss()

# Multiclass problem

In [14]:
class NeuralNet2:

    def __init__(self, input_size, hidden_size, n_classes):
        super(NeuralNet2, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        # no softmax required
        return out 
    

model = NeuralNet2(input_size=28*28, hidden_size=5, n_classes=3)
criterion = nn.CrossEntropyLoss()